# Diagnostics 1: Uncertainty Calibration and Selection Bias

See [here](notes/250902_next_steps.md) for more details. Essentially, I aim to run two diagnostic tests to see what's going wrong with the active learning campaign. Those are: Uncertainty calibration and selection bias. For uncertainty calibration, I'll be checking for correlation between the variances and the residuals. For selection bias, I'll be comparing the distribution of labels and predictions of actively selected versus randomly selected samples.

In [1]:
from scripts.data_utils import train_val_test_split
from scripts.config import (
    DATA_PATH, 
    SEQUENCE_COL, 
    SCORE_COL, 
    TOK_MODEL, 
    VAL_SPLIT,
    TEST_SPLIT,
    BATCH_SIZE,
    RANDOM_SEED,
)

training_pool, val_dataloader, test_dataloader = train_val_test_split(
    DATA_PATH,
    SEQUENCE_COL,
    SCORE_COL,
    TOK_MODEL,
    VAL_SPLIT,
    TEST_SPLIT,
    BATCH_SIZE,
    RANDOM_SEED
)

In [2]:
import torch
from torch.utils.data import Subset, DataLoader
from scripts.training import initialize_and_train_new_model

def run_standard_finetuning(
        n_samples,
        approach, 
        model_name,
        batch_size, 
        learning_rate, 
        weight_decay, 
        epochs, 
        training_pool, 
        val_dataloader,
        patience
        ):

    # get dataloader of random train data
    random_indices = torch.randperm(len(training_pool))[:n_samples].tolist()
    train_subset = Subset(training_pool, random_indices)
    train_dataloader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)

    # train model
    model, history = initialize_and_train_new_model(approach, model_name, learning_rate, weight_decay, epochs, train_dataloader, val_dataloader, patience, return_history=True)
    return model, train_dataloader, history

In [3]:
from pathlib import Path
import pandas as pd
import numpy as np

from scripts.acquisition import acquire_new_batch, train_bootstrapped_ensemble, get_variances
from scripts.training import test_model
from scripts.diagnostics import get_residuals, get_chosen_labels_and_preds


def get_bootstrapped_ensemble_learning_curves(
        n_samples,
        initial_n_samples,
        n_samples_per_batch,
        model_name, 
        approach,
        learning_rate, 
        weight_decay, 
        epochs, 
        training_pool, 
        train_dataloader_batch_size,
        pool_dataloader_batch_size,
        val_dataloader, 
        test_dataloader,
        patience=5,
        n_models=5,
        results_path="active_vs_standard_learning_curves.csv"
):
    results_path = Path(results_path)
    results_dir = results_path.parent
    results_dir.mkdir(parents=True, exist_ok=True)

    # Load existing results if the file exists, otherwise start with a fresh DataFrame.
    if results_path.exists():
        all_results_df = pd.read_csv(results_path)
    else:
        all_results_df = pd.DataFrame()
    
    total_pool_size = len(training_pool)
    unlabeled_indices = np.arange(total_pool_size)
    labeled_indices = np.array([], dtype=np.int64)

    ensemble_predictions = None
    current_cycle = 1
    total_cycles = int(np.ceil((n_samples-initial_n_samples)/n_samples_per_batch)) + 1
    
    while len(labeled_indices) < n_samples and len(unlabeled_indices) > 0:
        print(f"\nCycle {current_cycle}/{total_cycles}\n-------------------------------------------------")

        # on the first cycle, choose random samples of initial_n_samples size
        if ensemble_predictions is None:
            print(f"Choosing initial {initial_n_samples} samples randomly...")
            train_dataloader, pool_dataloader, labeled_indices, unlabeled_indices = acquire_new_batch(
                training_pool, train_dataloader_batch_size, pool_dataloader_batch_size, initial_n_samples, n_samples_per_batch, labeled_indices, unlabeled_indices, acquisition_scores=None
            )
        # each other time, use the n_samples_per_batch with acquisition scores to select
        else:
            scores = get_variances(ensemble_predictions, f"{str(results_dir)}/variances/variances{current_cycle}.csv")
            print(f"Selecting new data points...")
            train_dataloader, pool_dataloader, labeled_indices, unlabeled_indices = acquire_new_batch(
                training_pool, train_dataloader_batch_size, pool_dataloader_batch_size, initial_n_samples, n_samples_per_batch, labeled_indices, unlabeled_indices, acquisition_scores=scores
            )
        
        # give message when loop ends
        if len(unlabeled_indices) == 0:
            print("Unlabeled pool is empty. Proceeding to final model training.")
            break
        
        # evaluate active vs standard
        final_results = []

        # active
        print(f"\nTraining and evaluating model using {len(labeled_indices)} actively selected samples...")
        model_active = initialize_and_train_new_model(approach, model_name, learning_rate, weight_decay, epochs, train_dataloader, val_dataloader, patience, return_history=False)
        results_active = test_model(model_active, test_dataloader, return_results=True)
        results_active = {
            'changing_var': 'n_samples',
            'local_exp_idx': current_cycle-1,
            'value': len(labeled_indices),
            'training_method': 'active',
            **results_active
        }
        final_results.append(results_active)

        # run diagnostics
        get_residuals(model_active, pool_dataloader, f"{str(results_dir)}/residuals/residuals{current_cycle}.csv")
        get_chosen_labels_and_preds(model_active, train_dataloader, f"{str(results_dir)}/labels_and_preds/active_labels_and_preds{current_cycle}.csv")

        # standard
        print(f"\nTraining and evaluating model using {len(labeled_indices)} randomly selected samples...")
        model_standard, standard_train_dataloader, _ = run_standard_finetuning(len(labeled_indices), approach, model_name, train_dataloader_batch_size, learning_rate, weight_decay, epochs, training_pool, val_dataloader, patience)
        results_standard = test_model(model_standard, test_dataloader, return_results=True)
        results_standard = {
            'changing_var': 'n_samples',
            'local_exp_idx': current_cycle-1,
            'value': len(labeled_indices),
            'training_method': 'standard',
            **results_standard
        }
        final_results.append(results_standard)

        get_chosen_labels_and_preds(model_standard, standard_train_dataloader, f"{str(results_dir)}/labels_and_preds/standard_labels_and_preds{current_cycle}.csv")
        
        # save to disk each time to save progress
        results_df = pd.DataFrame(final_results)
        all_results_df = pd.concat([all_results_df, results_df], ignore_index=True)
        all_results_df.to_csv(results_path, index=False)
        print(f"Progress for experiment {current_cycle-1} appended to {results_path}")

        # if it's the last cycle, skip ensemble predictions
        if (current_cycle == total_cycles):
            print("Experiments complete.")
            break

        print("Starting ensemble training and pool evaluation...")
        ensemble_predictions = train_bootstrapped_ensemble(n_models, model_name, approach, learning_rate, weight_decay, epochs, labeled_indices, train_dataloader_batch_size, training_pool, pool_dataloader, val_dataloader, patience)
    
        current_cycle += 1
    return all_results_df

In [5]:
from scripts.config import (
    MODEL_NAME,
    APPROACH,
    LEARNING_RATE,
    WEIGHT_DECAY,
    EPOCHS,
    POOL_BATCH_SIZE,
    PATIENCE,
    N_MODELS,
)

get_bootstrapped_ensemble_learning_curves(
    n_samples=256,
    initial_n_samples=16,
    n_samples_per_batch=16,
    model_name=MODEL_NAME,
    approach=APPROACH,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    epochs=EPOCHS,
    training_pool=training_pool,
    train_dataloader_batch_size=BATCH_SIZE,
    pool_dataloader_batch_size=POOL_BATCH_SIZE,
    val_dataloader=val_dataloader,
    test_dataloader=test_dataloader,
    patience=PATIENCE,
    n_models=N_MODELS,
    results_path='results/06_uncertainty_calibration/active_vs_standard_learning_curve.csv'
)

/home/oscar/miniconda3/envs/plm-active-learning/lib/python3.12/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: Metric `SpearmanCorrcoef` will save all targets and predictions in the buffer. For large datasets, this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)



Cycle 1/16
-------------------------------------------------
Choosing initial 16 samples randomly...

Training and evaluating model using 16 actively selected samples...


[Training]:  64%|██████▍   | 32/50 [00:09<00:05,  3.55it/s]


Early stopping triggered after 10 epochs with no improvement.
Train Loss: 0.0134 | Val Loss: 0.2605 | SpearmanR: 0.2480


[Getting Residuals]: 100%|██████████| 25/25 [00:01<00:00, 16.88it/s]


...

/home/oscar/miniconda3/envs/plm-active-learning/lib/python3.12/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: Metric `SpearmanCorrcoef` will save all targets and predictions in the buffer. For large datasets, this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
[Training]:  96%|█████████▌| 48/50 [00:48<00:02,  1.01s/it]


Early stopping triggered after 10 epochs with no improvement.
Train Loss: 0.0229 | Val Loss: 0.1083 | SpearmanR: 0.7281


[Getting Chosen Labels and Preds]: 100%|██████████| 16/16 [00:00<00:00, 126.77it/s]


Saving residuals to results/06_uncertainty_calibration/labels_and_preds/standard_labels_and_preds16.csv...
Progress for experiment 15 appended to results/06_uncertainty_calibration/active_vs_standard_learning_curve.csv
Experiments complete.


,changing_var,local_exp_idx,value,training_method,avg_test_loss,spearmanr,pearsonr,final_mse
0,n_samples,0,16,active,0.198404,0.089504,0.138942,0.198380
1,n_samples,0,16,standard,0.222621,0.027158,0.016171,0.222519
2,n_samples,0,16,active,0.200590,0.076409,0.094881,0.200553
3,n_samples,0,16,standard,0.205489,-0.160188,-0.126538,0.205435
4,n_samples,0,16,active,0.199817,0.141446,0.154365,0.199759
5,n_samples,0,16,standard,0.197825,0.121573,0.128695,0.197800
6,n_samples,1,32,active,0.196137,0.101768,0.121738,0.196127
7,n_samples,1,32,standard,0.197525,-0.203042,-0.148789,0.197506
8,n_samples,0,16,active,0.275546,0.258759,0.384468,0.275646
9,n_samples,0,16,standard,0.239904,0.289643,0.289137,0.239860


## Analysis

In [59]:
import pandas as pd
import glob
from pathlib import Path

def assemble_df(attribute):
    files = glob.glob(f"results/06_uncertainty_calibration/{attribute}/*.csv")
    files.sort()

    all_dfs = []

    for filepath in files:
        temp_df = pd.read_csv(filepath)
        if attribute == 'labels_and_preds':
            filename = Path(filepath).stem
            training_type = 'active' if filename.startswith("active") else 'standard'
            num = filename[-1] if filename[-2].startswith("s") else filename[-2:]
            temp_df = temp_df.rename(columns={'labels': f'{training_type}_labels{num}',
                                              'preds': f'{training_type}_preds{num}'})
        elif attribute == 'variances':
            column_name = Path(filepath).stem
            temp_df = temp_df.rename(columns={'variance': column_name})
        else:
            column_name = Path(filepath).stem
            temp_df = temp_df.rename(columns={attribute: column_name})
        all_dfs.append(temp_df)

    final_df = pd.concat(all_dfs, axis=1)

    return final_df

In [64]:
import plotly.express as px

def plot_distributions(df, column_names, plotting_labels=True):
    melted_df = df[column_names].melt(
    var_name='Training Method',     
    value_name='Labels' if plotting_labels else 'Preds' 
)

    fig = px.histogram(
        data_frame=melted_df,
        x='Labels' if plotting_labels else 'Preds' ,                                  
        color='Training Method',                                 
        barmode='overlay',                             
        opacity=0.65,                                  
        histnorm='probability density',                
        title='Distribution of Variances Across Active Learning Cycles'
    )

    fig.show()

### Uncertainty

In [ ]:
variances_df = assemble_df('variances')
residuals_df = assemble_df('residuals')
merged_df = pd.concat([variances_df, residuals_df], axis=1)
merged_df

,variances10,variances11,variances12,variances13,variances14,variances15,variances16,variances2,variances3,variances4,...,residuals15,residuals16,residuals2,residuals3,residuals4,residuals5,residuals6,residuals7,residuals8,residuals9
0,0.010486,0.025583,0.048223,0.016065,0.009742,0.031411,0.028075,0.168271,0.105175,0.033408,...,0.117926,0.234689,0.240201,0.344249,0.547413,0.165809,0.425429,0.141126,0.070147,0.069537
1,0.042847,0.027055,0.021365,0.030832,0.056617,0.040480,0.102804,0.115111,0.039509,0.016360,...,0.442486,0.401448,0.850812,0.811963,0.345282,1.061635,0.918316,1.289529,1.085083,1.125114
2,0.013225,0.036619,0.067221,0.025856,0.025835,0.021028,0.004860,0.084911,0.010705,0.016902,...,0.138658,0.047270,0.069874,0.115371,0.252515,0.040035,0.290288,0.062190,0.097773,0.089653
3,0.018188,0.028687,0.001117,0.015263,0.025918,0.024833,0.046549,0.131170,0.134421,0.019372,...,0.373498,0.504986,0.278149,0.454235,0.188546,0.439912,0.319117,0.398082,0.627397,0.554399
4,0.011649,0.035836,0.035112,0.008232,0.022019,0.069180,0.013510,0.126485,0.033159,0.035262,...,0.278331,0.300524,0.412412,0.518250,0.471094,0.253817,0.529046,0.401245,0.322690,0.335143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.136989,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.077908,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3149,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.085897,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.126600,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
import plotly.express as px

pairs_to_plot = [('variances2', 'residuals2'),('variances6', 'residuals6'),('variances10', 'residuals10'),('variances14', 'residuals14')]
colors = ['blue', 'red', 'orange', 'purple']

for i, pair in enumerate(pairs_to_plot):
    fig = px.scatter(
        data_frame=merged_df,
        x=pair[0],
        y=pair[1],
        trendline='ols',
    )
    fig.update_traces(
    marker=dict(color=colors[i])
    )
    fig.show()

### Selection bias

In [60]:
labels_and_preds_df = assemble_df("labels_and_preds")
labels_and_preds_df

,active_labels1,active_preds1,active_labels10,active_preds10,active_labels11,active_preds11,active_labels12,active_preds12,active_labels13,active_preds13,...,standard_labels5,standard_preds5,standard_labels6,standard_preds6,standard_labels7,standard_preds7,standard_labels8,standard_preds8,standard_labels9,standard_preds9
0,-0.036354,-0.107666,0.088585,0.432001,1.167766,0.945142,0.284879,0.282786,0.641427,0.726994,...,-0.263598,-0.120962,-0.911415,-0.776029,0.158221,0.465098,-0.484527,-0.394888,0.067395,0.134904
1,-0.180261,-0.283052,0.258336,0.546001,0.607098,0.415099,0.462165,0.212660,0.062256,0.095716,...,0.146577,0.248761,1.243742,1.030968,-0.309355,-0.335464,-0.020560,0.134665,1.042827,0.901584
2,0.302464,0.344470,0.828947,0.858783,0.161135,0.109368,-0.756513,-0.685431,0.597890,0.630525,...,-0.446529,-0.356209,-0.122269,-0.208577,0.593735,0.614576,0.282405,0.260362,0.040280,0.084740
3,-0.990597,-0.407582,-0.180261,-0.581259,0.815583,0.607320,0.819739,0.664179,0.398955,0.398922,...,-0.049718,0.320475,-0.814505,-0.592489,-0.244630,-0.164468,0.424377,0.436567,-0.212445,0.006337
4,-0.475687,-0.379855,0.787555,0.873218,0.361586,0.191383,0.088585,0.016937,0.596740,0.521443,...,0.593735,0.634870,0.828947,0.849784,-0.756513,-0.394833,-0.580422,-0.448594,-0.350748,-0.463443
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
252,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
253,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
pairs_to_plot = [('active_labels2', 'standard_labels2'),('active_labels6', 'standard_labels6'),('active_labels10', 'standard_labels10'),('active_labels16', 'standard_labels16'),]

for pair in pairs_to_plot:
    plot_distributions(labels_and_preds_df, list(pair), plotting_labels=True)

In [73]:
import pandas as pd
import numpy as np
from scipy import stats

def check_normality(data: pd.Series):
    """
    Calculates skewness, kurtosis, and performs Shapiro-Wilk and Jarque-Bera
    normality tests on a pandas Series.

    Args:
        data (pd.Series): The column of data to test.
    """
    # --- 1. Clean data by removing NaN values ---
    data = data.dropna()
    
    if len(data) < 3:
        print(f"Column '{data.name}' has fewer than 3 non-NaN values. Cannot perform tests.")
        return

    print(f"--- Normality Report for column: '{data.name}' ---")

    # --- 2. Skewness and Kurtosis ---
    skewness = data.skew()
    kurtosis = data.kurt() # This is excess kurtosis (Fisher's definition)
    
    print(f"Skewness: {skewness:.3f}")
    print(f"Kurtosis: {kurtosis:.3f}\n")
    
    # Interpretation helpers
    if -0.5 < skewness < 0.5:
        print("Skewness indicates the distribution is fairly symmetrical.")
    elif -1 < skewness < -0.5 or 0.5 < skewness < 1:
        print("Skewness indicates the distribution is moderately skewed.")
    else:
        print("Skewness indicates the distribution is highly skewed.")

    if kurtosis > 1:
        print("Kurtosis indicates heavy tails (leptokurtic), suggesting outliers are more likely.")
    elif kurtosis < -1:
         print("Kurtosis indicates light tails (platykurtic), suggesting outliers are less likely.")
    else:
        print("Kurtosis is within a normal range.")

    # --- 3. Shapiro-Wilk Test ---
    # More powerful for smaller sample sizes (<5000)
    shapiro_stat, shapiro_p = stats.shapiro(data)
    print(f"\nShapiro-Wilk Test:")
    print(f"  - Statistic: {shapiro_stat:.3f}")
    print(f"  - p-value: {shapiro_p:.3f}")
    
    alpha = 0.05
    if shapiro_p > alpha:
        print("  - Conclusion: Sample looks Gaussian (fail to reject H0)")
    else:
        print("  - Conclusion: Sample does not look Gaussian (reject H0)")

    # --- 4. Jarque-Bera Test ---
    # Good for larger sample sizes, based on skew and kurtosis
    jb_stat, jb_p = stats.jarque_bera(data)
    print(f"\nJarque-Bera Test:")
    print(f"  - Statistic: {jb_stat:.3f}")
    print(f"  - p-value: {jb_p:.3f}")
    
    if jb_p > alpha:
        print("  - Conclusion: Sample looks Gaussian (fail to reject H0)")
    else:
        print("  - Conclusion: Sample does not look Gaussian (reject H0)")
    
    print("-" * 45 + "\n")

In [86]:
import pandas as pd
import numpy as np
from scipy import stats
import re

def get_normality_stats(data: pd.Series) -> dict:
    """
    Calculates normality statistics for a pandas Series and returns them as a dictionary.

    Args:
        data (pd.Series): The column of data to test.

    Returns:
        dict: A dictionary containing skewness, kurtosis, and test results.
    """
    # Clean data by removing NaN values
    data = data.dropna()
    
    # If not enough data points, return a dictionary of NaNs
    if len(data) < 3:
        return {
            'skewness': np.nan,
            'kurtosis': np.nan,
            'shapiro_stat': np.nan,
            'shapiro_p': np.nan,
            'jb_stat': np.nan,
            'jb_p': np.nan
        }

    # Calculate Skewness and Kurtosis
    skewness = data.skew()
    kurtosis = data.kurt()  # This is excess kurtosis (Fisher's definition)
    
    # Perform Shapiro-Wilk Test
    shapiro_stat, shapiro_p = stats.shapiro(data)

    # Perform Jarque-Bera Test
    jb_stat, jb_p = stats.jarque_bera(data)
    
    # Compile results into a dictionary
    results = {
        'skewness': skewness,
        'kurtosis': kurtosis,
        'shapiro_stat': shapiro_stat,
        'shapiro_p': shapiro_p,
        'jb_stat': jb_stat,
        'jb_p': jb_p
    }
    
    return results

def analyze_distributions(df: pd.DataFrame) -> pd.DataFrame:
    """
    Analyzes each column of a DataFrame for normality, parsing column names
    for metadata, and returns a results DataFrame.

    The expected column name pattern is '{training_type}_{labels_or_preds}{cycle_number}',
    e.g., 'active_preds10' or 'standard_labels2'.

    Args:
        df (pd.DataFrame): The DataFrame with distributions to analyze.

    Returns:
        pd.DataFrame: A new DataFrame containing the analysis results.
    """
    all_results = []
    
    # Regex to separate the text part from the number part (e.g., 'preds' from '10')
    name_pattern = re.compile(r'([a-zA-Z]+)(\d+)')

    for col_name in df.columns:
        # --- 1. Parse column name for metadata ---
        try:
            parts = col_name.split('_')
            training_type = parts[0]
            
            match = name_pattern.match(parts[1])
            if not match:
                print(f"Warning: Skipping column '{col_name}' as it doesn't match the expected pattern.")
                continue
            
            labels_or_preds, cycle_number = match.groups()
            cycle_number = int(cycle_number)

        except (IndexError, AttributeError):
            print(f"Warning: Skipping column '{col_name}' due to parsing error.")
            continue
            
        # --- 2. Get normality statistics ---
        stats_dict = get_normality_stats(df[col_name])
        
        # --- 3. Combine metadata and stats ---
        row = {
            'training_type': training_type,
            'labels_or_preds': labels_or_preds,
            'cycle_number': cycle_number,
            **stats_dict  # Unpack the stats dictionary into the row
        }
        all_results.append(row)
        
    # --- 4. Create the final results DataFrame ---
    results_df = pd.DataFrame(all_results)
    return results_df

In [89]:
normality_df = analyze_distributions(labels_and_preds_df)
normality_df

,training_type,labels_or_preds,cycle_number,skewness,kurtosis,shapiro_stat,shapiro_p,jb_stat,jb_p
0,active,labels,1,-0.060358,-0.803503,0.967776,8.014238e-01,0.580105,0.748224
1,active,preds,1,0.462770,-1.738309,0.800923,2.801483e-03,2.159460,0.339687
2,active,labels,10,-0.266843,-0.823128,0.973667,3.758096e-03,6.510293,0.038575
3,active,preds,10,-0.398019,-1.477378,0.861344,5.529655e-11,18.529505,0.000095
4,active,labels,11,-0.251245,-0.821583,0.975012,2.940630e-03,6.900424,0.031739
...,...,...,...,...,...,...,...,...,...
59,standard,preds,7,0.154458,-0.785730,0.980333,9.783797e-02,3.451447,0.178044
60,standard,labels,8,0.330137,-0.296798,0.985384,1.868077e-01,2.858080,0.239539
61,standard,preds,8,0.281074,-0.880159,0.965748,2.521545e-03,5.895844,0.052449
62,standard,labels,9,-0.099319,-0.606375,0.983723,8.601145e-02,2.589865,0.273916


In [91]:
normality_df.to_csv('results/06_uncertainty_calibration/normality_results.csv', index=False)

In [92]:
def plot_normality_trends(results_df: pd.DataFrame):
    """
    Generates several plots to visualize normality metrics over training cycles.

    Args:
        results_df (pd.DataFrame): The DataFrame containing the normality analysis results.
    """
    # Sort the dataframe to ensure lines are plotted correctly
    results_df = results_df.sort_values(by=['training_type', 'labels_or_preds', 'cycle_number'])

    # --- Plot 1: Skewness over Cycles ---
    # This plot helps visualize if distributions become more or less symmetrical.
    # A value near 0 is symmetrical.
    fig_skew = px.line(
        results_df,
        x='cycle_number',
        y='skewness',
        color='training_type',
        facet_row='labels_or_preds',
        title='Skewness of Distributions Over Training Cycles',
        labels={'cycle_number': 'Training Cycle', 'skewness': 'Skewness'},
        markers=True
    )
    # Add a reference line for perfect symmetry
    fig_skew.add_hline(y=0, line_dash="dot", line_color="black", annotation_text="Symmetrical")
    fig_skew.show()

    # --- Plot 2: Kurtosis over Cycles ---
    # This plot shows the "tailedness" of the distributions.
    # A value near 0 is similar to a normal distribution.
    # Positive values mean heavier tails (more outliers).
    fig_kurt = px.line(
        results_df,
        x='cycle_number',
        y='kurtosis',
        color='training_type',
        facet_row='labels_or_preds',
        title='Kurtosis of Distributions Over Training Cycles',
        labels={'cycle_number': 'Training Cycle', 'kurtosis': 'Excess Kurtosis'},
        markers=True
    )
    # Add a reference line for normal kurtosis
    fig_kurt.add_hline(y=0, line_dash="dot", line_color="black", annotation_text="Normal Tails")
    fig_kurt.show()

    # --- Plot 3: Shapiro-Wilk P-value over Cycles ---
    # This is the most direct test for normality.
    # If p-value < 0.05, the distribution is likely not normal.
    fig_pval = px.line(
        results_df,
        x='cycle_number',
        y='shapiro_p',
        color='training_type',
        facet_row='labels_or_preds',
        title='Shapiro-Wilk P-value Over Training Cycles',
        labels={'cycle_number': 'Training Cycle', 'shapiro_p': 'P-value'},
        markers=True
    )
    # Add the significance level alpha=0.05 as a reference line
    fig_pval.add_hline(y=0.05, line_dash="dash", line_color="red", annotation_text="α = 0.05")
    fig_pval.show()

In [93]:
plot_normality_trends(normality_df)

In [94]:
all_df = pd.read_csv('avrpikC_full.csv')
all_df

,aa_sequence,enrichment_score
0,GLKQKIVIKVAMEGNNCRSKAMALVASTGGVDSVALVGDLRDKIEV...,1.468796
1,GLKRIIVIKVAREGNNCRSKAMALVASTGGVDSVALVGDLRGKIEV...,1.415944
2,GLKRIIVIKVAMEGNNCRSKAMALVASTGGVDSVALVGDLRGKIEV...,1.389615
3,GLKQKIVIKVAMEGNNCRSKAMALVASTGGVDSVALVGDLRDKIEV...,1.359651
4,GLKQKIVIKVAMEGNNCRSKAMALVASTGGVDSVALVGDLRGKIEV...,1.343857
...,...,...
3955,GLKQKIVIKVAMEGNNCRSKAMALVASTGGVDSVALVGDLRDKIEV...,-1.041749
3956,GLKQKIVIKVAMEGNNCRSKAMALVASTGGVDSVALVGDLRDKIEA...,-1.041749
3957,GLKQKIVIKVAMEGNNCRSKAMALVASTGGVDSVALVGDLRDKIEV...,-1.057543
3958,GLKQKIVIKVAMEGNNCRSKAMALVASTGGVDSVALVGDLRDKTEV...,-1.057543


In [96]:
fig = px.histogram(
        data_frame=all_df,
        x='enrichment_score',                                                                                         
        histnorm='probability density',                
        title='Enrichment scores'
    )

fig.show()

In [97]:
all_df.enrichment_score.mean()

np.float64(0.09718174617570516)